# Работа с CustomTorch

Этот ноутбук представляет собой мануал по работе с библиотекой CusTorch

Данная библиотека является легковестным гибридом Keras и PyTorch и реализует базовые методы построения собственных нейронных сетей

## Начало работы

## Ознакомление с базовым функционалом

### Импорт
Для начала работы импортируем все классы и методы из библиотеки

In [73]:
from CustomTorch import *


Также импортируем крайне важную библиоетку для работы с векторами

In [74]:
import numpy as np

### класс Tensor

Данный класс является обёрткой для повторяющихся математическх операций над векорами, а также хранит промежуточные значения, такие как градиент

In [75]:
# Класс Tensor инициализируется через numpy.array
example_tensor_1 = Tensor(np.array([[1, 2], [3, 4]]))
example_tensor_2 = Tensor(np.array([[4, 3], [2, 1]]))

Мы можем производить различные операции над векторами

In [76]:
# Матричное умножение
new_tensor = example_tensor_1.__matmul__(example_tensor_2)
display(new_tensor.data)

# Сложение
new_tensor = example_tensor_1.__add__(example_tensor_2)
display(new_tensor.data)

array([[ 8,  5],
       [20, 13]])

array([[5, 5],
       [5, 5]])

## Класс Linear

Представляет собой реализацию линейного слоя нейронной сети

In [77]:
# Инициализация класса c параметрами: (размерность входящего тензора, размерность выхода)
Linear_Example = Linear(4, 2)

Методы работы с линейным слоем

In [78]:
# Прохождение слоя
result = Linear_Example.forward(Tensor([1, 2, 3, 4]))
result.data

array([-0.00512157,  0.03539639])

In [ ]:
# Обратное распространение ошибки

# Создаём желаемый ответ
true_result = Tensor([5.1, 2.3])

# Создаём класс функции потерь и считаем Loss
loss_func = MSELoss()
grad = loss_func.forward(result, true_result)


In [80]:
# Производим обратное распространение 
backward_result = Linear_Example.backward(grad)
backward_result.data

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)